In [1]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>",
  root.dir = './'
)
#knitr::opts_chunk$set(eval = FAslice2E)

This vignette shows how to apply CellChat to identify major signaling changes across different biological conditions by quantitative contrasts and joint manifold learning. We showcase CellChat’s diverse functionalities by applying it to a scRNA-seq data on cells from two biological conditions: noslice1esional (slice1, normal) and lesional (slice2, diseased) human skin from patients with atopic dermatitis. **These two datasets (conditions) have the same cell population compositions after joint clustering. If there are slightly or vastly different cell population compositions between different datasets, please check out another related tutorial. **

CellChat employs a top-down approach, i.e., starting with the big picture and then refining it in a greater detail on the signaling mechanisms, to identify signaling changes at different levels, including both general principles of cell-cell communication and dysfunctional cell populations/signaling pathways/ligand-receptors.

## Load the required libraries

In [1]:
Sys.setenv(RETICULATE_PYTHON = "/home/chrissy1/.conda/envs/cellchat_env/bin/python")
library(reticulate)
py_config()

## Load the required libraries
library(CellChat)
package.version("CellChat")
library(patchwork)
options(stringsAsFactors = FALSE)
library(ggplot2)
library(dplyr)


python:         /home/chrissy1/.conda/envs/cellchat_env/bin/python
libpython:      /home/chrissy1/.conda/envs/cellchat_env/lib/libpython3.11.so
pythonhome:     /home/chrissy1/.conda/envs/cellchat_env:/home/chrissy1/.conda/envs/cellchat_env
version:        3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
numpy:          /home/chrissy1/.conda/envs/cellchat_env/lib/python3.11/site-packages/numpy
numpy_version:  1.26.4

NOTE: Python version was forced by RETICULATE_PYTHON

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: ggplot2



[1] "2.1.2"

# Load CellChat object of each dataset and then merge together

Users need to run CellChat on each dataset seperately and then merge different CellChat objects together.
Please do `updateCellChat` if you have CellChat objects that are obtained using the earlier version (< 1.6.0).

In [2]:
cellchat_dir = '/home/chrissy1/spatial/stomics/ovary_froz/redo/seurat/bin50_processed/cellchat/'
base.fig.dir <- paste0(cellchat_dir, 'comparison')
dir.create(base.fig.dir, showWarnings = FALSE)
key = 'Vitri'  # fresh  vitri slow
cellchat.vitri= readRDS(paste0(cellchat_dir, key, '_endo_subbed.rds'))
key = 'Slow'  # fresh  vitri slow
cellchat.slow = readRDS(paste0(cellchat_dir, key, '_endo_subbed.rds'))
key = 'Fresh'  # fresh  vitri slow
cellchat.fresh = readRDS(paste0(cellchat_dir, key, '_endo_subbed.rds'))


celltype_OI = sapply(unique(cellchat.fresh@meta$annot), function(x) {
    if (!grepl("ST(", x, fixed = TRUE)) {
        return(x)
    } else {
        return(NA)
    }
}) 
celltype_OI = celltype_OI[!is.na(celltype_OI)]


In [3]:
celltype_OI = sapply(as.character(unique(cellchat.fresh@meta$annot)), function(x)
    if (!grepl("ST(", x, fixed = TRUE)) {
        return(x)
    } else {
        return(NA)}
    , simplify = "array") %>% .[!is.na(.)]
celltype_OI

ST_LIP(INSL3)    B_Plasma(IGKC)         ENDO_SM_1       FIB(COL1A1) 
  "ST_LIP(INSL3)"  "B_Plasma(IGKC)"       "ENDO_SM_1"     "FIB(COL1A1)" 
        ENDO_SM_5         ENDO_SM_4       ST_SM(MT1A)          O(PDCD5) 
      "ENDO_SM_5"       "ENDO_SM_4"     "ST_SM(MT1A)"        "O(PDCD5)" 
    ST_LIP(GSTA1)         ENDO_SM_0    ST_SM(ADAMTS4)         ENDO_SM_3 
  "ST_LIP(GSTA1)"       "ENDO_SM_0"  "ST_SM(ADAMTS4)"       "ENDO_SM_3" 
        ENDO_SM_2     ERY_ENDO(HBB)   B_Plasma(IGLL5)   B_Plasma(IGLC2) 
      "ENDO_SM_2"   "ERY_ENDO(HBB)" "B_Plasma(IGLL5)" "B_Plasma(IGLC2)" 
        ENDO_SM_6 
      "ENDO_SM_6"

## Check numerical stats

In [4]:
df <- list(vitri=subsetCommunication(cellchat.vitri), slow=subsetCommunication(cellchat.slow), fresh=subsetCommunication(cellchat.fresh))


In [5]:
print('union of all samples')
union(union(unique(df$fresh$pathway_name), unique(df$slow$pathway_name)), unique(df$vitri$pathway_name))
print('in all samples')
all = intersect(intersect(unique(df$fresh$pathway_name), unique(df$slow$pathway_name)), unique(df$vitri$pathway_name))
all
print('fresh and slow')
setdiff(intersect(unique(df$fresh$pathway_name), unique(df$slow$pathway_name)), all)
print('fresh and vitri')
setdiff(intersect(unique(df$fresh$pathway_name), unique(df$vitri$pathway_name)), all)
print('slow and vitri')
setdiff(intersect(unique(df$slow$pathway_name), unique(df$vitri$pathway_name)), all)


# pathways unique to each slice
print('pathways unique to fresh ')
setdiff(unique(df$fresh$pathway_name), union(unique(df$slow$pathway_name), unique(df$vitri$pathway_name)))
print('pathways unique to slow ')
setdiff(unique(df$slow$pathway_name), union(unique(df$fresh$pathway_name), unique(df$vitri$pathway_name)))
print('pathways unique to vitri')
setdiff(unique(df$vitri$pathway_name), union(unique(df$fresh$pathway_name), unique(df$slow$pathway_name)))

[1] "union of all samples"


[1] "TGFb"            "BMP"             "WNT"             "ncWNT"          
 [5] "EGF"             "NRG"             "FGF"             "PDGF"           
 [9] "VEGF"            "IGF"             "CCL"             "CXCL"           
[13] "MIF"             "TWEAK"           "VISFATIN"        "ANGPTL"         
[17] "ANGPT"           "MK"              "PTN"             "NT"             
[21] "SEMA3"           "CALCR"           "GAS"             "GRN"            
[25] "PROS"            "PTPR"            "IGFBP"           "CypA"           
[29] "COLLAGEN"        "FN1"             "LAMININ"         "RELN"           
[33] "THBS"            "TENASCIN"        "HSPG"            "27HC"           
[37] "Androstenedione" "Androsterone"    "Cholesterol"     "Desmosterol"    
[41] "Testosterone"    "APP"             "CADM"            "CD46"           
[45] "CD99"            "CDH"             "CDH5"            "CLDN"           
[49] "CSPG4"           "EPHA"            "EPHB"            "ESAM"           
[53] "JAM"             "MAG"             "MPZ"             "NCAM"           
[57] "NECTIN"          "NEGR"            "NOTCH"           "NRXN"           
[61] "PECAM1"          "PTPRM"           "SEMA4"           "SEMA6"          
[65] "SEMA7"           "VCAM"            "VISTA"           "GAP"            
[69] "PCDH"            "ADGRA"           "ADGRE"           "ADGRG"          
[73] "ADGRL"           "FLRT"            "UNC5"            "CNTN"           
[77] "Netrin"          "LIFR"            "SPP1"            "SLIT"           
[81] "AGRN"            "Prostaglandin"   "RA"              "CD34"           
[85] "SELE"            "EDN"             "DHEA"

[1] "in all samples"


[1] "TGFb"            "WNT"             "EGF"             "NRG"            
 [5] "FGF"             "PDGF"            "VEGF"            "IGF"            
 [9] "CCL"             "CXCL"            "MIF"             "TWEAK"          
[13] "VISFATIN"        "ANGPTL"          "ANGPT"           "MK"             
[17] "PTN"             "SEMA3"           "GAS"             "GRN"            
[21] "PROS"            "PTPR"            "IGFBP"           "CypA"           
[25] "COLLAGEN"        "FN1"             "LAMININ"         "THBS"           
[29] "TENASCIN"        "HSPG"            "Androstenedione" "Cholesterol"    
[33] "Desmosterol"     "Testosterone"    "APP"             "CADM"           
[37] "CD46"            "CD99"            "CDH"             "CDH5"           
[41] "CLDN"            "CSPG4"           "EPHA"            "EPHB"           
[45] "ESAM"            "JAM"             "MPZ"             "NCAM"           
[49] "NECTIN"          "NEGR"            "NOTCH"           "NRXN"           
[53] "PECAM1"          "PTPRM"           "SEMA4"           "SEMA6"          
[57] "VCAM"            "GAP"             "PCDH"            "ADGRA"          
[61] "ADGRE"           "ADGRG"           "ADGRL"           "FLRT"           
[65] "UNC5"            "CNTN"            "Netrin"

[1] "fresh and slow"


...

[1] "RELN"         "27HC"         "Androsterone" "MAG"          "SEMA7"       
[6] "VISTA"

[1] "pathways unique to slow "


[1] "LIFR" "SLIT" "AGRN"

[1] "pathways unique to vitri"


[1] "EDN"  "DHEA"

In [6]:
print('union of all samples')
union(union(unique(df$fresh$interaction_name), unique(df$slow$interaction_name)), unique(df$vitri$interaction_name))
print('in all samples')
all = intersect(intersect(unique(df$fresh$interaction_name), unique(df$slow$interaction_name)), unique(df$vitri$interaction_name))
all
print('fresh and slow')
setdiff(intersect(unique(df$fresh$interaction_name), unique(df$slow$interaction_name)), all)
print('fresh and vitri')
setdiff(intersect(unique(df$fresh$interaction_name), unique(df$vitri$interaction_name)), all)
print('slow and vitri')
setdiff(intersect(unique(df$slow$interaction_name), unique(df$vitri$interaction_name)), all)


# pathways unique to each slice
print('pathways unique to fresh ')
setdiff(unique(df$fresh$interaction_name), union(unique(df$slow$interaction_name), unique(df$vitri$interaction_name)))
print('pathways unique to slow ')
setdiff(unique(df$slow$interaction_name), union(unique(df$fresh$interaction_name), unique(df$vitri$interaction_name)))
print('pathways unique to vitri')
setdiff(unique(df$vitri$interaction_name), union(unique(df$fresh$interaction_name), unique(df$slow$interaction_name)))

[1] "union of all samples"


[1] "TGFB1_TGFBR1_TGFBR2"                     
  [2] "TGFB2_TGFBR1_TGFBR2"                     
  [3] "TGFB1_ACVR1_TGFBR1"                      
  [4] "TGFB2_ACVR1_TGFBR1"                      
  [5] "BMP6_ACVR1_BMPR2"                        
  [6] "BMP6_BMPR1A_BMPR2"                       
  [7] "BMP6_BMPR1B_BMPR2"                       
  [8] "WNT4_FZD1_LRP5"                          
  [9] "WNT4_FZD3_LRP5"                          
 [10] "WNT4_FZD1_LRP6"                          
 [11] "WNT4_FZD3_LRP6"                          
 [12] "WNT4_FZD7_LRP6"                          
 [13] "WNT4_FZD8_LRP6"                          
 [14] "WNT5B_FZD1"                              
 [15] "WNT5B_FZD3"                              
 [16] "WNT5B_FZD7"                              
 [17] "WNT5B_FZD8"                              
 [18] "AREG_EGFR"                               
 [19] "AREG_EGFR_ERBB2"                         
 [20] "NRG1_ERBB4"                              
 [21] "FGF2_FGFR1"                              
 [22] "FGF2_FGFR2"                              
 [23] "FGF7_FGFR1"                              
 [24] "FGF7_FGFR2"                              
 [25] "PDGFB_PDGFRA"                            
 [26] "PDGFB_PDGFRB"                            
 [27] "PDGFC_PDGFRA"                            
 [28] "PDGFD_PDGFRB"                            
 [29] "VEGFB_VEGFR1"                            
 [30] "VEGFC_VEGFR2"                            
 [31] "PGF_VEGFR1"                              
 [32] "IGF1_IGF1R"                              
 [33] "IGF2_IGF1R"                              
 [34] "IGF2_IGF2R"                              
 [35] "CCL2_ACKR1"                              
 [36] "CCL5_ACKR1"                              
 [37] "CCL14_ACKR1"                             
 [38] "CXCL8_ACKR1"                             
 [39] "CXCL12_CXCR4"                            
 [40] "MIF_CD74_CXCR4"                          
 [41] "MIF_CD74_CD44"                           
 [42] "TNFSF12_TNFRSF12A"                       
 [43] "NAMPT_ITGA5_ITGB1"                       
 [44] "ANGPTL1_ITGA1_ITGB1"                     
 [45] "ANGPTL2_ITGA5_ITGB1"                     
 [46] "ANGPT2_TEK"                              
 [47] "ANGPT2_ITGA5_ITGB1"                      
 [48] "MDK_SDC2"                                
 [49] "MDK_SDC4"                                
 [50] "MDK_ITGA6_ITGB1"                         
 [51] "MDK_LRP1"                                
 [52] "MDK_NCL"                                 
 [53] "PTN_SDC2"                                
 [54] "PTN_SDC4"                                
 [55] "PTN_NCL"                                 
 [56] "NTF3_NTRK2"                              
 [57] "NTF3_NTRK3"                              
 [58] "SEMA3A_NRP1_PLXNA2"                      
 [59] "SEMA3A_NRP1_PLXNA4"                      
 [60] "SEMA3C_NRP1_PLXNA2"                      
 [61] "SEMA3C_NRP1_PLXNA4"                      
 [62] "SEMA3C_NRP2_PLXNA2"                      
 [63] "SEMA3C_NRP2_PLXNA4"                      
 [64] "SEMA3G_NRP2_PLXNA2"                      
 [65] "SEMA3G_NRP2_PLXNA4"                      
 [66] "SEMA3C_NRP1_NRP2"                        
 [67] "SEMA3C_PLXND1"                           
 [68] "ADM_CALCRL"                              
 [69] "GAS6_AXL"                                
 [70] "GAS6_TYRO3"                              
 [71] "GRN_SORT1"                               
 [72] "PROS1_AXL"                               
 [73] "PROS1_TYRO3"                             
 [74] "PTPRS_NTRK3"                             
 [75] "IGFBP3_TMEM219"                          
 [76] "PPIA_BSG"                                
 [77] "COL1A1_ITGA1_ITGB1"                      
 [78] "COL1A2_ITGA1_ITGB1"                      
 [79] "COL4A1_ITGA1_ITGB1"                      
 [80] "COL4A2_ITGA1_ITGB1"                      
 [81] "COL4A5_ITGA1_ITGB1"                      
 [82] "COL6A1_ITGA1_ITGB1"       

[1] "in all samples"


[1] "TGFB1_TGFBR1_TGFBR2"                  
  [2] "TGFB2_TGFBR1_TGFBR2"                  
  [3] "TGFB1_ACVR1_TGFBR1"                   
  [4] "TGFB2_ACVR1_TGFBR1"                   
  [5] "AREG_EGFR"                            
  [6] "NRG1_ERBB4"                           
  [7] "FGF2_FGFR1"                           
  [8] "FGF7_FGFR1"                           
  [9] "PDGFC_PDGFRA"                         
 [10] "PDGFD_PDGFRB"                         
 [11] "VEGFB_VEGFR1"                         
 [12] "PGF_VEGFR1"                           
 [13] "IGF2_IGF1R"                           
 [14] "IGF2_IGF2R"                           
 [15] "CCL2_ACKR1"                           
 [16] "CCL14_ACKR1"                          
 [17] "MIF_CD74_CD44"                        
 [18] "TNFSF12_TNFRSF12A"                    
 [19] "NAMPT_ITGA5_ITGB1"                    
 [20] "ANGPTL2_ITGA5_ITGB1"                  
 [21] "ANGPT2_TEK"                           
 [22] "ANGPT2_ITGA5_ITGB1"                   
 [23] "MDK_SDC2"                             
 [24] "MDK_SDC4"                             
 [25] "MDK_ITGA6_ITGB1"                      
 [26] "MDK_LRP1"                             
 [27] "MDK_NCL"                              
 [28] "PTN_SDC2"                             
 [29] "PTN_SDC4"                             
 [30] "PTN_NCL"                              
 [31] "SEMA3C_NRP1_PLXNA2"                   
 [32] "SEMA3C_NRP1_PLXNA4"                   
 [33] "SEMA3C_NRP2_PLXNA2"                   
 [34] "SEMA3C_NRP2_PLXNA4"                   
 [35] "SEMA3G_NRP2_PLXNA2"                   
 [36] "SEMA3G_NRP2_PLXNA4"                   
 [37] "SEMA3C_NRP1_NRP2"                     
 [38] "SEMA3C_PLXND1"                        
 [39] "GAS6_AXL"                             
 [40] "GAS6_TYRO3"                           
 [41] "GRN_SORT1"                            
 [42] "PROS1_AXL"                            
 [43] "PROS1_TYRO3"                          
 [44] "PTPRS_NTRK3"                          
 [45] "IGFBP3_TMEM219"                       
 [46] "PPIA_BSG"                             
 [47] "COL1A1_ITGA1_ITGB1"                   
 [48] "COL1A2_ITGA1_ITGB1"                   
 [49] "COL4A1_ITGA1_ITGB1"                   
 [50] "COL4A2_ITGA1_ITGB1"                   
 [51] "COL6A1_ITGA1_ITGB1"                   
 [52] "COL6A2_ITGA1_ITGB1"                   
 [53] "COL6A3_ITGA1_ITGB1"                   
 [54] "FN1_ITGA3_ITGB1"                      
 [55] "FN1_ITGA5_ITGB1"                      
 [56] "LAMA2_ITGA1_ITGB1"                    
 [57] "LAMA4_ITGA1_ITGB1"                    
 [58] "LAMA5_ITGA1_ITGB1"                    
 [59] "LAMB1_ITGA1_ITGB1"                    
 [60] "LAMB2_ITGA1_ITGB1"                    
 [61] "LAMC1_ITGA1_ITGB1"                    
 [62] "LAMA2_ITGA3_ITGB1"                    
 [63] "LAMA4_ITGA3_ITGB1"                    
 [64] "LAMA5_ITGA3_ITGB1"                    
 [65] "LAMB1_ITGA3_ITGB1"                    
 [66] "LAMB2_ITGA3_ITGB1"                    
 [67] "LAMC1_ITGA3_ITGB1"                    
 [68] "THBS1_ITGA3_ITGB1"                    
 [69] "THBS2_ITGA3_ITGB1"                    
 [70] "COL1A1_ITGA3_ITGB1"                   
 [71] "COL1A2_ITGA3_ITGB1"                   
 [72] "COL4A1_ITGA3_ITGB1"                   
 [73] "COL4A2_ITGA3_ITGB1"                   
 [74] "COL6A1_ITGA3_ITGB1"                   
 [75] "COL6A2_ITGA3_ITGB1"                   
 [76] "COL6A3_ITGA3_ITGB1"                   
 [77] "LAMA2_ITGA6_ITGB1"                    
 [78] "LAMA4_ITGA6_ITGB1"                    
 [79] "LAMA5_ITGA6_ITGB1"                    
 [80] "LAMB1_ITGA6_ITGB1"                    
 [81] "LAMB2_ITGA6_ITGB1"                    
 [82] "LAMC1_ITGA6_ITGB1"                    
 [83] "LAMA2_ITGA7_ITGB1"                    
 [84] "LAMA4_ITGA7_ITGB1"                    
 [85] "LAMA5_ITGA7_ITGB1"                    
 [86] "LAMB1_ITGA7_ITGB1"                    
 [87] "LAMB2_ITGA7_ITGB1"                    


[1] "fresh and slow"


...

[1] "WNT4_FZD1_LRP5"                         
 [2] "WNT4_FZD3_LRP5"                         
 [3] "WNT4_FZD1_LRP6"                         
 [4] "WNT4_FZD3_LRP6"                         
 [5] "WNT4_FZD7_LRP6"                         
 [6] "WNT4_FZD8_LRP6"                         
 [7] "WNT5B_FZD1"                             
 [8] "WNT5B_FZD3"                             
 [9] "WNT5B_FZD7"                             
[10] "WNT5B_FZD8"                             
[11] "AREG_EGFR_ERBB2"                        
[12] "FGF2_FGFR2"                             
[13] "FGF7_FGFR2"                             
[14] "PDGFB_PDGFRA"                           
[15] "PDGFB_PDGFRB"                           
[16] "CCL5_ACKR1"                             
[17] "CXCL12_CXCR4"                           
[18] "MIF_CD74_CXCR4"                         
[19] "ANGPTL1_ITGA1_ITGB1"                    
[20] "LAMA1_ITGA1_ITGB1"                      
[21] "LAMA1_ITGA3_ITGB1"                      
[22] "RELN_ITGA3_ITGB1"                       
[23] "LAMA1_ITGA6_ITGB1"                      
[24] "LAMA1_ITGA7_ITGB1"                      
[25] "LAMA1_ITGA9_ITGB1"                      
[26] "LAMA1_ITGAV_ITGB8"                      
[27] "LAMA3_ITGAV_ITGB8"                      
[28] "LAMA1_CD44"                             
[29] "LAMA1_DAG1"                             
[30] "RELN_VLDLR"                             
[31] "27Hydroxycholesterol-27HC-CYP11A1_NR1H4"
[32] "ANDROSTERONE-HSD17B6_NR1H4"             
[33] "CDH2_CDH2"                              
[34] "EFNB1_EPHB1"                            
[35] "EFNB1_EPHB2"                            
[36] "EFNB2_EPHB1"                            
[37] "MAG_MAG"                                
[38] "DLK1_NOTCH4"                            
[39] "DLL4_NOTCH1"                            
[40] "DLL4_NOTCH2"                            
[41] "DLL4_NOTCH3"                            
[42] "DLL4_NOTCH4"                            
[43] "JAG1_NOTCH4"                            
[44] "SEMA7A_ITGB1_ITGA1"                     
[45] "SEMA7A_PLXNC1"                          
[46] "VSIR_IGSF11"                            
[47] "APP_SORL1"                              
[48] "CD99_PILRB"

[1] "pathways unique to slow "


[1] "BMP6_ACVR1_ACVR2A"                       
 [2] "WNT2B_FZD1_LRP5"                         
 [3] "WNT2B_FZD6_LRP5"                         
 [4] "WNT2B_FZD7_LRP5"                         
 [5] "WNT2B_FZD8_LRP5"                         
 [6] "WNT8A_FZD6_LRP5"                         
 [7] "WNT2B_FZD6_LRP6"                         
 [8] "WNT8A_FZD6_LRP6"                         
 [9] "WNT5B_FZD6"                              
[10] "NRG3_ERBB4"                              
[11] "FGF2_FGFR4"                              
[12] "VEGFA_VEGFR1R2"                          
[13] "CCL8_ACKR1"                              
[14] "CXCL12_ACKR3"                            
[15] "MIF_ACKR3"                               
[16] "CTF1_LIFR_IL6ST"                         
[17] "ANGPTL2_TLR4"                            
[18] "SEMA3C_NRP1_PLXNA1"                      
[19] "SEMA3C_NRP2_PLXNA1"                      
[20] "SEMA3F_NRP2_PLXNA1"                      
[21] "SEMA3F_NRP2_PLXNA4"                      
[22] "GAS6_MERTK"                              
[23] "SLIT2_ROBO1"                             
[24] "SLIT2_ROBO2"                             
[25] "PROS1_MERTK"                             
[26] "THBS4_ITGA3_ITGB1"                       
[27] "COL6A6_ITGA9_ITGB1"                      
[28] "LAMC2_ITGA9_ITGB1"                       
[29] "LAMC2_ITGAV_ITGB8"                       
[30] "COL6A6_ITGAV_ITGB8"                      
[31] "COL6A6_CD44"                             
[32] "COL6A6_SDC4"                             
[33] "THBS4_SDC4"                              
[34] "LAMA2_SV2A"                              
[35] "LAMA4_SV2A"                              
[36] "LAMB2_SV2A"                              
[37] "LAMC1_SV2A"                              
[38] "LAMC2_SV2A"                              
[39] "THBS4_CD36"                              
[40] "THBS4_CD47"                              
[41] "AGRN_DAG1"                               
[42] "LAMC2_DAG1"                              
[43] "PGF2a-AKR1C3_PTGFR"                      
[44] "PGF2a-CBR1_PTGFR"                        
[45] "RetinoicAcid-RA-ALDH1A1_RARA_CRABP2"     
[46] "RetinoicAcid-RA-ALDH1A1_RARA_RXRA_CRABP2"
[47] "RetinoicAcid-RA-ALDH1A1_RXRA_CRABP2"     
[48] "RetinoicAcid-RA-ALDH1A2_RARA_CRABP2"     
[49] "RetinoicAcid-RA-ALDH1A2_RARA_RXRA_CRABP2"
[50] "RetinoicAcid-RA-ALDH1A2_RXRA_CRABP2"     
[51] "RetinoicAcid-RA-ALDH1A3_RARA_CRABP2"     
[52] "RetinoicAcid-RA-ALDH1A3_RARA_RXRA_CRABP2"
[53] "RetinoicAcid-RA-ALDH1A3_RXRA_CRABP2"     
[54] "CNTN1_NRCAM"                             
[55] "NCAM1_NCAM2"                             
[56] "JAG2_NOTCH1"                             
[57] "JAG2_NOTCH2"                             
[58] "JAG2_NOTCH3"                             
[59] "SEMA4D_PLXNB1"                           
[60] "SEMA4D_PLXNB2"                           
[61] "SEMA4G_PLXNB2"                           
[62] "FLRT2_UNC5B"                             
[63] "LRFN5_PTPRD"                             
[64] "LRFN5_PTPRF"                             
[65] "LRFN5_PTPRS"                             
[66] "NRXN1_LRRTM4"                            
[67] "NRXN3_LRRTM4"                            
[68] "LRRTM4_PTPRS"                            
[69] "NTN4_UNC5B"

[1] "pathways unique to vitri"


[1] "TGFB3_TGFBR1_TGFBR2"                 
 [2] "TGFB3_ACVR1_TGFBR1"                  
 [3] "SPP1_ITGA8_ITGB1"                    
 [4] "ANGPTL4_ITGA5_ITGB1"                 
 [5] "ANGPTL4_CDH11"                       
 [6] "ANGPTL4_SDC2"                        
 [7] "ANGPTL4_SDC4"                        
 [8] "EDN1_EDNRA"                          
 [9] "EDN1_EDNRB"                          
[10] "PGE2-PTGES3_PTGER2"                  
[11] "RetinoicAcid-RA-ALDH1A1_RORB"        
[12] "RetinoicAcid-RA-ALDH1A2_RORB"        
[13] "RetinoicAcid-RA-ALDH1A3_RORB"        
[14] "Dehydroepiandrosterone-DHEA-STS_ESR1"
[15] "CDH1_CDH1"                           
[16] "FLRT3_ADGRL2"                        
[17] "FLRT3_ADGRL3"                        
[18] "FLRT3_FLRT3"                         
[19] "FLRT3_UNC5C"                         
[20] "NRXN2_CLSTN1"                        
[21] "NRXN2_CLSTN2"                        
[22] "NRXN1_LRRTM3"

## Form different comparison sets

In [7]:
group.new = union(levels(cellchat.slow@idents), levels(cellchat.vitri@idents))
cellchat.sl <- liftCellChat(cellchat.slow, group.new)
cellchat.vi <- liftCellChat(cellchat.vitri, group.new)

frozen.list <- list(vitri= cellchat.vi, slow = cellchat.sl)
frozen.pathways <- intersect(cellchat.vi@netP$pathways, cellchat.sl@netP$pathways)

group.new = levels(cellchat.fresh@idents)
cellchat.vitri<- liftCellChat(cellchat.vitri, group.new)
cellchat.slow <- liftCellChat(cellchat.slow, group.new)

freshslow.list <- list(fresh = cellchat.fresh, slow = cellchat.slow)
freshslow.pathways <- intersect(cellchat.fresh@netP$pathways, cellchat.slow@netP$pathways)

freshvitri.list <- list(fresh = cellchat.fresh, vitri= cellchat.vitri)
freshvitri.pathways <- intersect(cellchat.fresh@netP$pathways, cellchat.vitri@netP$pathways)

object.list <- list(fresh = cellchat.fresh, slow = cellchat.slow, vitri= cellchat.vitri)

cellchat.frozen <- mergeCellChat(frozen.list, add.names = names(frozen.list))
cellchat.freshvitri<- mergeCellChat(freshvitri.list, add.names = names(freshvitri.list))
cellchat.freshslow <- mergeCellChat(freshslow.list, add.names = names(freshslow.list))
cellchat <- mergeCellChat(object.list, add.names = names(object.list))

The CellChat object will be lifted up using the cell labels B_Plasma(IGKC), B_Plasma(IGLC2), B_Plasma(IGLL5), ENDO_SM_0, ENDO_SM_1, ENDO_SM_2, ENDO_SM_3, ENDO_SM_4, ENDO_SM_5, ENDO_SM_6, ERY_ENDO(HBB), FIB(COL1A1), O(PDCD5), ST_LIP(GSTA1), ST_LIP(INSL3), ST_SM(ADAMTS4), ST_SM(MT1A), ST(ARL15), ST(BEX2), ST(BNC2), ST(EXT1), ST(FNDC3B), ST(KRT19), ST(MAGI1), ST(MAML2), ST(PRKG1), ST(RAD51B), ST(RBFOX1), ST(RPL41), ST(SECTM1), ST(SLCO5A1), ST(ZFPM2)



Update slots object@net, object@netP, object@idents in a single dataset... 


The CellChat object will be lifted up using the cell labels B_Plasma(IGKC), B_Plasma(IGLC2), B_Plasma(IGLL5), ENDO_SM_0, ENDO_SM_1, ENDO_SM_2, ENDO_SM_3, ENDO_SM_4, ENDO_SM_5, ENDO_SM_6, ERY_ENDO(HBB), FIB(COL1A1), O(PDCD5), ST_LIP(GSTA1), ST_LIP(INSL3), ST_SM(ADAMTS4), ST_SM(MT1A), ST(ARL15), ST(BEX2), ST(BNC2), ST(EXT1), ST(FNDC3B), ST(KRT19), ST(MAGI1), ST(MAML2), ST(PRKG1), ST(RAD51B), ST(RBFOX1), ST(RPL41), ST(SECTM1), ST(SLCO5A1), ST(ZFPM2)



Update slots object@net, object@netP, object@idents in a single dataset... 


The CellChat object will be lifted up using the cell labels B_Plasma(IGKC), B_Plasma(IGLC2), B_Plasma(IGLL5), ENDO_SM_0, ENDO_SM_1, ENDO_SM_2, ENDO_SM_3, ENDO_SM_4, ENDO_SM_5, ENDO_SM_6, ERY_ENDO(HBB), FIB(COL1A1), O(PDCD5), ST_LIP(GSTA1), ST_LIP(INSL3), ST_SM(ADAMTS4), ST_SM(MT1A), ST(ARL15), ST(BEX2), ST(BNC2), ST(EXT1), ST(FNDC3B), ST(KRT19), ST(MAGI1), ST(MAML2), ST(PRKG1), ST(RAD51B), ST(RBFOX1), ST(RPL41), ST(SECTM1), ST(SLCO5A1), ST(ZFPM2)



Update slots object@net, object@netP, object@idents in a single dataset... 


The CellChat object will be lifted up using the cell labels B_Plasma(IGKC), B_Plasma(IGLC2), B_Plasma(IGLL5), ENDO_SM_0, ENDO_SM_1, ENDO_SM_2, ENDO_SM_3, ENDO_SM_4, ENDO_SM_5, ENDO_SM_6, ERY_ENDO(HBB), FIB(COL1A1), O(PDCD5), ST_LIP(GSTA1), ST_LIP(INSL3), ST_SM(ADAMTS4), ST_SM(MT1A), ST(ARL15), ST(BEX2), ST(BNC2), ST(EXT1), ST(FNDC3B), ST(KRT19), ST(MAGI1), ST(MAML2), ST(PRKG1), ST(RAD51B), ST(RBFOX1), ST(RPL41), ST(SECTM1), ST(SLCO5A1), ST(ZFPM2)



Update slots object@net, object@netP, object@idents in a single dataset... 


Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.

Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.

Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.

Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.



** Change here and run all below to generate comparison figures between chosen conditions

In [8]:
cch = cellchat.frozen
pathways = frozen.pathways
cch.list = frozen.list
samples = paste0(names(cch.list), collapse = '.')
fig.dir = paste0(base.fig.dir, '/', samples, '/')
dir.create(fig.dir, showWarnings = FALSE)

df.net <- subsetCommunication(cch)
vert.rec = which(unique(cch@idents[[1]]) %in% celltype_OI)

# Part I: Predict general principles of cell-cell communication

CellChat starts with the big picture to predict general principles of cell-cell communication. When comparing cell-cell communication among multiple biological conditions, it can answer the following biological questions:

* Whether the cell-cell communication is enhanced or not

* The interaction between which cell types is significantly changed

* How the major sources and targets change from one condition to another

## Compare the total number of interactions and interaction strength

To answer on question on whether the cell-cell communication is enhanced or not, CellChat compares the the total number of interactions and interaction strength of the inferred cell-cell communication networks from different biological conditions.

In [10]:
setwd(fig.dir)
pdf(paste0('ttl_interactions.pdf'), width = 10, height = 10)
gg1 <- compareInteractions(cellchat, show.legend = F, group = c(1,2,3))
gg2 <- compareInteractions(cellchat, show.legend = F, group = c(1,2,3), measure = "weight")
gg1 + gg2
dev.off()

png 
  2

## Compare the number of interactions and interaction strength among different cell populations

To identify the interaction between which cell populations showing significant changes, CellChat compares the number of interactions and interaction strength among different cell populations.

### Differential number of interactions or interaction strength among different cell populations
The differential number of interactions or interaction strength in the cell-cell communication network between two datasets can be visualized using circle plot, where $\color{red}{\text{red}}$ (or $\color{blue}{\text{blue}}$) colored edges represent $\color{red}{\text{increased}}$ (or $\color{blue}{\text{decreased}}$) signaling in the second dataset compared to the first one.

In [11]:
table(cch@meta$batch)


Vitri  Slow 
46364 45796 

In [12]:
setwd(fig.dir)
f = paste0(samples, '.diffInteraction_circle.pdf')
pdf(f, width = 10, height = 10)
par(mfrow = c(1,length(cch.list)))
g1 = netVisual_diffInteraction(cch, weight.scale = T, vertex.label.cex=0.7)
g2 = netVisual_diffInteraction(cch, weight.scale = T, measure = "weight", vertex.label.cex=0.7)
dev.off()
# ggsave(filename = f, width = 10, height = 10)

png 
  2

We can also show differential number of interactions or interaction strength in a greater details using a heatmap. The top colored bar plot represents the sum of column of values displayed in the heatmap (incoming signaling). The right colored bar plot represents the sum of row of values (outgoing signaling). In the colorbar, $\color{red}{\text{red}}$ (or $\color{blue}{\text{blue}}$) represents $\color{red}{\text{increased}}$ (or $\color{blue}{\text{decreased}}$) signaling in the second dataset compared to the first one.

In [13]:
f = paste0(samples, '.diffInteraction_heatmap_cnts.pdf')
pdf(f, width = 10, height = 10)
netVisual_heatmap(cch)
dev.off()

Do heatmap based on a merged object 




png 
  2

In [14]:
f = paste0(samples, '.diffInteraction_heatmap_weights.pdf')
pdf(f, width = 10, height = 10)
netVisual_heatmap(cch, measure = "weight")
dev.off()

Do heatmap based on a merged object 




png 
  2

## Compare the major sources and targets in 2D space

Comparing the outgoing and incoming interaction strength in 2D space allows ready identification of the cell populations with significant changes in sending or receiving signals between different datasets.

In [15]:
num.link <- sapply(frozen.list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
weight.MinMax <- c(min(num.link), max(num.link)) # control the dot size in the different datasets

pdf(paste0(samples, '.diffInteraction_scatter.pdf'), width = 10, height = 5)
gg <- list()
for (i in 1:length(cch.list)) {
  gg[[i]] <- netAnalysis_signalingRole_scatter(cch.list[[i]], title = names(cch.list)[i], weight.MinMax = weight.MinMax, )
}
patchwork::wrap_plots(plots = gg)
dev.off()

Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways



Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways



png 
  2

In [16]:
# ## Identify signaling changes associated with one cell group
# for (celltype in levels(cch@idents[[1]])) {
#     f = paste0(samples, '.', celltype, '.diffInteraction_scatter.pdf')
#     tryCatch({
#         pdf(f, width = 10, height = 5)
#         gg1 = netAnalysis_signalingChanges_scatter(cch, idents.use = celltype)
#         gg2 = netAnalysis_signalingChanges_scatter(cch, idents.use = celltype)
#         patchwork::wrap_plots(plots = c(gg1,gg2))
#         dev.off()
#         # ggsave(filename = f, plot=gg1+gg2, width = 10, height = 5)
#     }, error = function(e) {
#         print(e)
#     })
# }

# Part II: Identify altered signaling with distinct network architecture and interaction strength

CellChat then can identify signaling networks with larger (or less) difference, signaling groups, and the conserved and context-specific signaling pathways based on their cell-cell communication networks among multiple biological conditions.

## Identify signaling networks with larger (or less) difference as well as signaling groups based on their functional/structure similarity

CellChat performs joint manifold learning and classification of the inferred communication networks based on their functional and topological similarity. NB: Such analysis is applicable to more than two datasets.

**Functional similarity**: High degree of functional similarity indicates major senders and receivers are similar, and it can be interpreted as the two signaling pathways or two ligand-receptor pairs exhibit similar and/or redundant roles. **NB**: Functional similarity analysis is not applicable to multiple datsets with different cell type composition.

**Structural similarity**: A structural similarity was used to compare their signaling network structure, without considering the similarity of senders and receivers. **NB**: Structural similarity analysis is applicable to multiple datsets with the same cell type composition or the vastly different cell type composition.

Here we can run the manifold and classification learning analysis based on the functional similarity because the two datasets have the the same cell type composition.

## Identify and visualize the conserved and context-specific signaling pathways

By comparing the information flow/interaction strengh of each signaling pathway, we can identify signaling pathways, (i) turn off, (ii) decrease, (iii) turn on or (iv) increase, by change their information flow at one condition as compared to another condition.

### Compare the overall information flow of each signaling pathway
We can identify the conserved and context-specific signaling pathways by simply comparing the information flow for each signaling pathway, which is defined by the sum of communication probability among all pairs of cell groups in the inferred network (i.e., the total weights in the network).

This bar graph can be plotted in a stacked mode or not. Significant signaling pathways were ranked based on differences in the overall information flow within the inferred networks between slice1 and slice2 skin. The top signaling pathways colored red are enriched in slice1 skin, and these colored green were enriched in the slice2 skin.

In [9]:
setwd(fig.dir)
pdf(paste0(samples, '.ranknet.pdf'), width = 10, height = 10)
# rankNet(cch, mode = "comparison", stacked = T, do.stat = TRUE)
rankNet(cch, mode = "comparison", stacked = F, do.stat = TRUE)
dev.off()

png 
  2

### Compare outgoing (or incoming) signaling associated with each cell population
The above analysis summarize the information from the outgoing and incoming signaling together. We can also compare the outgoing (or incoming) signaling pattern between two datasets, allowing to identify signaling pathways/ligand-receptors that exhibit different signaling patterns.

In [18]:
library(ComplexHeatmap)
setwd(fig.dir)
if (length(cch.list)==3) {
    pathway.union <- union(union(cch.list[[1]]@netP$pathways, cch.list[[2]]@netP$pathways), cch.list[[3]]@netP$pathways)
} else if (length(cch.list)==2) {
    pathway.union <- union(cch.list[[1]]@netP$pathways, cch.list[[2]]@netP$pathways)
}

title = names(cch.list)[[1]]
pdf(paste0(samples, '.signalingRole_heatmap_outgoing_', title,'.pdf'), width = 14, height = 14)
netAnalysis_signalingRole_heatmap(cch.list[[title]], pattern = "outgoing", signaling = pathway.union, title = title, width = 14, height = 14)
dev.off()
pdf(paste0(samples, '.signalingRole_heatmap_incoming_', title,'.pdf'), width = 14, height = 14)
netAnalysis_signalingRole_heatmap(cch.list[[title]], pattern = "incoming", signaling = pathway.union, title = title, width = 14, height = 14, color.heatmap = "GnBu")
dev.off()
pdf(paste0(samples, '.signalingRole_heatmap_all', title,'.pdf'), width = 14, height = 14)
netAnalysis_signalingRole_heatmap(cch.list[[title]], pattern = "all", signaling = pathway.union, title = title, width = 14, height = 14, color.heatmap = "OrRd")
dev.off()
title = names(cch.list)[[2]]
pdf(paste0(samples, '.signalingRole_heatmap_outgoing_', title,'.pdf'), width = 14, height = 14)
netAnalysis_signalingRole_heatmap(cch.list[[title]], pattern = "outgoing", signaling = pathway.union, title = title, width = 14, height = 14)
dev.off()
pdf(paste0(samples, '.signalingRole_heatmap_incoming_', title,'.pdf'), width = 14, height = 14)
netAnalysis_signalingRole_heatmap(cch.list[[title]], pattern = "incoming", signaling = pathway.union, title = title, width = 14, height = 14, color.heatmap = "GnBu")
dev.off()
pdf(paste0(samples, '.signalingRole_heatmap_all', title,'.pdf'), width = 14, height = 14)
netAnalysis_signalingRole_heatmap(cch.list[[title]], pattern = "all", signaling = pathway.union, title = title, width = 14, height = 14, color.heatmap = "OrRd")
dev.off()


Loading required package: grid

ComplexHeatmap version 2.14.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))




png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

# Part III: Identify the upgulated and down-regulated signaling ligand-receptor pairs

## Identify dysfunctional signaling by comparing the communication probabities

We can compare the communication probabilities mediated by ligand-receptor pairs from some cell groups to other cell groups. This can be done by setting `comparison` in the function `netVisual_bubble`.

Moreover, we can identify the upgulated (increased) and down-regulated (decreased) signaling ligand-receptor pairs in one dataset compared to the other dataset. This can be done by specifying `max.dataset` and `min.dataset` in the function `netVisual_bubble`. The increased signaling means these signaling have higher communication probability (strength) in one dataset compared to the other dataset.

all cell types

In [19]:
setwd(fig.dir)
all_celltypes = unique(cch@idents[[1]])
setwd(fig.dir)
for (i in 1:length(all_celltypes)) {  # length(all_celltypes)
    f1 = paste0(samples, '.signalingRole_heatmap_as_source_', all_celltypes[i],'_increased.pdf')
    f2 = paste0(samples, '.signalingRole_heatmap_as_target_', all_celltypes[i],'_increased.pdf')
    tryCatch({
        g = netVisual_bubble(cch, sources.use = all_celltypes[i], comparison = c(1, 2), max.dataset = 2, 
            title.name = paste0("Increased signaling in ",names(cch.list)[2]), angle.x = 45, remove.isolate = T, color.text.use = F)
        ggsave(filename = f1, plot=g, width = 10, height = 10)
        
        g = netVisual_bubble(cch, targets.use = all_celltypes[i],  comparison = c(1, 2), max.dataset = 2, 
            title.name = paste0("Increased signaling in ",names(cch.list)[2]), angle.x = 45, remove.isolate = T, color.text.use = F)
        ggsave(filename = f2, plot=g, width = 10, height = 10)
    }, error = function(e) {
        print(e)
    })
}

setwd(fig.dir)
all_celltypes = unique(cch@idents[[1]])
setwd(fig.dir)
for (i in 1:length(all_celltypes)) {  # length(all_celltypes)
    f1 = paste0(samples, '.signalingRole_heatmap_as_source_', all_celltypes[i],'_decreased.pdf')
    f2 = paste0(samples, '.signalingRole_heatmap_as_target_', all_celltypes[i],'_decreased.pdf')
    tryCatch({
        g = netVisual_bubble(cch, sources.use = all_celltypes[i], comparison = c(1, 2), max.dataset = 1, 
            title.name = paste0("Increased signaling in ",names(cch.list)[2]), angle.x = 45, remove.isolate = T, color.text.use = F)
        ggsave(filename = f1, plot=g, width = 10, height = 10)
        
        g = netVisual_bubble(cch, targets.use = all_celltypes[i],  comparison = c(1, 2), max.dataset = 1, 
            title.name = paste0("Increased signaling in ",names(cch.list)[2]), angle.x = 45, remove.isolate = T, color.text.use = F)
        ggsave(filename = f2, plot=g, width = 10, height = 10)
    }, error = function(e) {
        print(e)
    })
}


Comparing communications on a merged object 




Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 




<simpleError in `$<-.data.frame`(`*tmp*`, "source.target", value = character(0)): replacement has 0 rows, data has 1>


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing com

<simpleError in `$<-.data.frame`(`*tmp*`, "source.target", value = character(0)): replacement has 0 rows, data has 1>


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing com

all available pathways

In [20]:
unique(unlist(sapply(cellchat@netP, function(x) {x$pathways})))

[1] "COLLAGEN"        "MK"              "LAMININ"         "APP"            
 [5] "FLRT"            "FN1"             "Cholesterol"     "CLDN"           
 [9] "CD99"            "ADGRL"           "CypA"            "IGF"            
[13] "IGFBP"           "NOTCH"           "PTPRM"           "GAP"            
[17] "PDGF"            "THBS"            "PTN"             "GAS"            
[21] "NCAM"            "ADGRA"           "JAM"             "VCAM"           
[25] "BMP"             "FGF"             "MIF"             "NRXN"           
[29] "TENASCIN"        "CD46"            "CNTN"            "SEMA3"          
[33] "HSPG"            "ESAM"            "CDH5"            "VEGF"           
[37] "PTPR"            "Androstenedione" "ADGRG"           "CDH"            
[41] "MPZ"             "ADGRE"           "PECAM1"          "TWEAK"          
[45] "NEGR"            "Testosterone"    "SEMA4"           "GRN"            
[49] "SEMA6"           "Desmosterol"     "NECTIN"          "EPHA"           
[53] "CXCL"            "Netrin"          "PCDH"            "PROS"           
[57] "UNC5"            "ANGPT"           "ANGPTL"          "CCL"            
[61] "EPHB"            "RELN"            "VISFATIN"        "27HC"           
[65] "SEMA7"           "CALCR"           "NT"              "CADM"           
[69] "ncWNT"           "TGFb"            "CSPG4"           "MAG"            
[73] "Androsterone"    "EGF"             "WNT"             "VISTA"          
[77] "NRG"             "RA"              "SELE"            "SPP1"           
[81] "Prostaglandin"   "AGRN"            "SLIT"            "CD34"           
[85] "LIFR"            "EDN"             "DHEA"

In [21]:
setwd(fig.dir)
all_pathways = unique(unlist(sapply(cch@netP, function(x) {x$pathways})))

for (i in 1:length(all_pathways)) {  # length(all_celltypes)
    f1 = paste0(samples, '.signalingRole_heatmap_as_source_', all_pathways[i],'_increased.pdf')
    tryCatch({
        g = netVisual_bubble(cch, signaling = all_pathways[i], comparison = c(1, 2), max.dataset = 2, 
            title.name = paste0("Increased signaling in ",names(cch.list)[2]), angle.x = 45, remove.isolate = T, color.text.use = F)
        ggsave(filename = f1, plot=g, width = 10, height = 10)        
    }, error = function(e) {
        print(e)
    })
}
for (i in 1:length(all_celltypes)) {  # length(all_celltypes)
    f1 = paste0(samples, '.signalingRole_heatmap_as_source_', all_pathways[i],'_decreased.pdf')
    tryCatch({
        g = netVisual_bubble(cch, signaling = all_pathways[i], comparison = c(1, 2), max.dataset = 1, 
            title.name = paste0("Increased signaling in ",names(cch.list)[2]), angle.x = 45, remove.isolate = T, color.text.use = F)
        ggsave(filename = f1, plot=g, width = 10, height = 10)
        
    }, error = function(e) {
        print(e)
    })
}


Comparing communications on a merged object 




Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing com

<simpleError in subsetCommunication_internal(net, LR, cells.level, slot.name = slot.name,     sources.use = sources.use, targets.use = targets.use, signaling = signaling,     pairLR.use = pairLR.use, thresh = thresh, datasets = datasets,     ligand.pvalues = ligand.pvalues, ligand.logFC = ligand.logFC,     ligand.pct.1 = ligand.pct.1, ligand.pct.2 = ligand.pct.2,     receptor.pvalues = receptor.pvalues, receptor.logFC = receptor.logFC,     receptor.pct.1 = receptor.pct.1, receptor.pct.2 = receptor.pct.2): No significant signaling interactions are inferred based on the input!>


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 




<simpleError in subsetCommunication_internal(net, LR, cells.level, slot.name = slot.name,     sources.use = sources.use, targets.use = targets.use, signaling = signaling,     pairLR.use = pairLR.use, thresh = thresh, datasets = datasets,     ligand.pvalues = ligand.pvalues, ligand.logFC = ligand.logFC,     ligand.pct.1 = ligand.pct.1, ligand.pct.2 = ligand.pct.2,     receptor.pvalues = receptor.pvalues, receptor.logFC = receptor.logFC,     receptor.pct.1 = receptor.pct.1, receptor.pct.2 = receptor.pct.2): No significant signaling interactions are inferred based on the input!>


...

Comparing communications on a merged object 




<simpleError in subsetCommunication_internal(net, LR, cells.level, slot.name = slot.name,     sources.use = sources.use, targets.use = targets.use, signaling = signaling,     pairLR.use = pairLR.use, thresh = thresh, datasets = datasets,     ligand.pvalues = ligand.pvalues, ligand.logFC = ligand.logFC,     ligand.pct.1 = ligand.pct.1, ligand.pct.2 = ligand.pct.2,     receptor.pvalues = receptor.pvalues, receptor.logFC = receptor.logFC,     receptor.pct.1 = receptor.pct.1, receptor.pct.2 = receptor.pct.2): No significant signaling interactions are inferred based on the input!>


Comparing communications on a merged object 




<simpleError in subsetCommunication_internal(net, LR, cells.level, slot.name = slot.name,     sources.use = sources.use, targets.use = targets.use, signaling = signaling,     pairLR.use = pairLR.use, thresh = thresh, datasets = datasets,     ligand.pvalues = ligand.pvalues, ligand.logFC = ligand.logFC,     ligand.pct.1 = ligand.pct.1, ligand.pct.2 = ligand.pct.2,     receptor.pvalues = receptor.pvalues, receptor.logFC = receptor.logFC,     receptor.pct.1 = receptor.pct.1, receptor.pct.2 = receptor.pct.2): No significant signaling interactions are inferred based on the input!>


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing communications on a merged object 


Comparing com

NB: The ligand-receptor pairs shown in the bubble plot can be accessed via `signaling.slice2Increased = gg1$data`.

## Identify differential signaling

The above method for identifying the upgulated and down-regulated signaling is perfomed by comparing the communication probability between two datasets for each L-R pair and each pair of cell groups. Alternative, we can identify the upgulated and down-regulated signaling ligand-receptor pairs based on the differential gene expression analysis. Specifically, we perform differential expression analysis between two biological conditions (i.e., slice1 and slice2) for each cell group, and then obtain the upgulated and down-regulated signaling based on the fold change of ligands in the sender cells and receptors in the receiver cells. Such analysis can be done as follows.

In [13]:
netMappingDEG = function (object, features.name, thresh = 0.05) 
{
    features.name <- paste0(features.name, ".info")
    if (!(features.name %in% names(object@var.features))) {
        stop("The input features.name does not exist in `names(object@var.features)`. Please first run `identifyOverExpressedGenes`! ")
    }
    DEG <- object@var.features[[features.name]]
    geneInfo <- object@DB$geneInfo
    complex_input <- object@DB$complex
    df.net <- subsetCommunication(object, thresh = thresh)
    if (is.list(df.net)) {
        net <- data.frame()
        for (ii in 1:length(df.net)) {
            df.net[[ii]]$datasets <- names(df.net)[ii]
            net <- rbind(net, df.net[[ii]])
        }
    }
    else {
        net <- df.net
    }
    net$source.ligand <- paste0(net$source, ".", net$ligand)
    net$target.receptor <- paste0(net$target, ".", net$receptor)
    DEG$clusters.features <- paste0(DEG$clusters, ".", DEG$features)
    net <- cbind(net, data.frame(ligand.pvalues = NA, ligand.logFC = NA, 
        ligand.pct.1 = NA, ligand.pct.2 = NA, receptor.pvalues = NA, 
        receptor.logFC = NA, receptor.pct.1 = NA, receptor.pct.2 = NA))
    idx1.ligand <- net$ligand %in% geneInfo$Symbol
    idx2.ligand <- which((net$ligand %in% geneInfo$Symbol) == 
        "FALSE")
    idx.pos <- match(net$source.ligand, DEG$clusters.features)
    idx1.source.ligand <- which(!is.na(idx.pos))
    idx1.clusters.features <- idx.pos[!is.na(idx.pos)]
    idx2.source.ligand <- which(idx1.ligand & !(net$source.ligand %in% 
        DEG$clusters.features))
    net[idx1.source.ligand, c("ligand.pvalues", "ligand.logFC", 
        "ligand.pct.1", "ligand.pct.2")] <- DEG[idx1.clusters.features, 
        c("pvalues", "logFC", "pct.1", "pct.2")]
    if (length(idx2.ligand) > 0) {
        net.temp.all <- data.frame()
        for (i in 1:length(idx2.ligand)) {
            complex <- net$ligand[idx2.ligand[i]]
            complexsubunits <- dplyr::select(complex_input[match(complex, 
                rownames(complex_input), nomatch = 0), ], starts_with("subunit"))
            complexsubunitsV <- unlist(complexsubunits)
            complexsubunitsV <- unique(complexsubunitsV[complexsubunitsV != 
                ""])
            source.ligand.complex <- paste0(net$source[idx2.ligand[i]], 
                ".", complexsubunitsV)
            idx.pos <- match(source.ligand.complex, DEG$clusters.features)
            idx1.clusters.features <- idx.pos[!is.na(idx.pos)]
            if (length(idx1.clusters.features) > 0) {
                net.temp <- DEG[idx1.clusters.features, c("pvalues", 
                  "logFC", "pct.1", "pct.2")]
                net.temp <- colMeans(net.temp, na.rm = TRUE)
                net.temp <- as.data.frame(t(net.temp))
                colnames(net.temp) <- c("ligand.pvalues", "ligand.logFC", 
                  "ligand.pct.1", "ligand.pct.2")
            }
            else {
                net.temp <- data.frame(ligand.pvalues = NA, ligand.logFC = NA, 
                  ligand.pct.1 = NA, ligand.pct.2 = NA)
            }
            net.temp.all <- rbind(net.temp.all, net.temp)
        }
        net[idx2.ligand, c("ligand.pvalues", "ligand.logFC", 
            "ligand.pct.1", "ligand.pct.2")] <- net.temp.all
    }
    idx1.receptor <- net$receptor %in% geneInfo$Symbol
    idx2.receptor <- which((net$receptor %in% geneInfo$Symbol) == 
        "FALSE")
    idx.pos <- match(net$target.receptor, DEG$clusters.features)
    idx1.target.receptor <- which(!is.na(idx.pos))
    idx1.clusters.features <- idx.pos[!is.na(idx.pos)]
    net[idx1.target.receptor, c("receptor.pvalues", "receptor.logFC", 
        "receptor.pct.1", "receptor.pct.2")] <- DEG[idx1.clusters.features, 
        c("pvalues", "logFC", "pct.1", "pct.2")]
    if (length(idx2.receptor) > 0) {
        net.temp.all <- data.frame()
        for (i in 1:length(idx2.receptor)) {
            complex <- net$receptor[idx2.receptor[i]]
            complexsubunits <- dplyr::select(complex_input[match(complex, 
                rownames(complex_input), nomatch = 0), ], starts_with("subunit"))
            complexsubunitsV <- unlist(complexsubunits)
            complexsubunitsV <- unique(complexsubunitsV[complexsubunitsV != 
                ""])
            target.receptor.complex <- paste0(net$target[idx2.receptor[i]], 
                ".", complexsubunitsV)
            idx.pos <- match(target.receptor.complex, DEG$clusters.features)
            idx1.clusters.features <- idx.pos[!is.na(idx.pos)]
            if (length(idx1.clusters.features) > 0) {
                net.temp <- DEG[idx1.clusters.features, c("pvalues", 
                  "logFC", "pct.1", "pct.2")]
                net.temp <- colMeans(net.temp, na.rm = TRUE)
                net.temp <- as.data.frame(t(net.temp))
                colnames(net.temp) <- c("receptor.pvalues", "receptor.logFC", 
                  "receptor.pct.1", "receptor.pct.2")
            }
            else {
                net.temp <- data.frame(receptor.pvalues = NA, 
                  receptor.logFC = NA, receptor.pct.1 = NA, receptor.pct.2 = NA)
            }
            net.temp.all <- rbind(net.temp.all, net.temp)
        }
        net[idx2.receptor, c("receptor.pvalues", "receptor.logFC", 
            "receptor.pct.1", "receptor.pct.2")] <- net.temp.all
    }
    return(net)
}
# define a positive dataset, i.e., the dataset with positive fold change against the other dataset
pos.dataset = names(cch.list)[2]
# define a char name used for storing the results of differential expression analysis
features.name = paste0(pos.dataset)
# perform differential expression analysis
# Of note, compared to CellChat version < v2, CellChat v2 now performs an ultra-fast Wilcoxon test using the presto package, which gives smaller values of logFC.
#  Thus we here set a smaller value of thresh.fc compared to the original one (thresh.fc = 0.1). Users can also provide a vector and dataframe of customized DEGs by modifying the cellchat@var.features$slice2 and cellchat@var.features$slice2.info.
cch <- identifyOverExpressedGenes(cch, pos.dataset = pos.dataset, 
            features.name = features.name, only.pos = FALSE, thresh.pc = 0.1, thresh.fc = 0.03, thresh.p = 0.8)
# map the results of differential expression analysis onto the inferred cell-cell communications to easily manage/subset the ligand-receptor pairs of interest
net <- netMappingDEG(cch, features.name = features.name)
write.table(net, file = paste0(fig.dir, 'net.txt'), sep = '\t', quote = F)


# extract the ligand-receptor pairs with upregulated ligands in slice2
tryCatch({
    net.up <- subsetCommunication(cch, net = net, datasets = pos.dataset, ligand.logFC = 0.1, thresh=0.05, receptor.logFC=0.1)
    write.table(net.up, file = paste0(fig.dir, 'net.up.txt'), sep = '\t', quote = F)
}, error = function(e) {
    print(e)
    net.up = NULL
})


# tryCatch({
#     # extract the ligand-receptor pairs with upregulated ligands and upregulated recetptors in slice1, i.e.,downregulated in slice2
#     net.down <- subsetCommunication(cch, net = net, datasets = pos.dataset, ligand.logFC = -0.1, thresh=0.05, receptor.logFC=-0.1)
#     write.table(net.down, file = paste0(fig.dir, 'net.down.txt'), sep = '\t', quote = F)
# }, error = function(e) {
#     print(e)
#     net.down = NULL
# })

# head(net.down[net.down$pathway_name == 'COLLAGEN',], 10)  # -0.14987451	
# head(net.up[net.up$pathway_name == 'COLLAGEN',], 10)  # 0.14747501	
# head(net.up[net.up$target == 'endothelial_perivascular(IGFBP7)',], 10)	
# head(net.up[net.up$source == 'endothelial_perivascular(IGFBP7)',], 10)	
# head(net.down[net.down$target == 'endothelial_perivascular(IGFBP7)',], 10)	
# head(net.down[net.down$source == 'endothelial_perivascular(IGFBP7)',], 10)

if (!is.null(net.up)) {
    gene.up <- extractGeneSubsetFromPair(net.up, cch)
    write.table(gene.up, file = paste0(fig.dir, 'gene.up.txt'), sep = '\t', quote = F)
}
gene.up

Use the joint cell labels from the merged CellChat object



[1] "CCL2"   "CCL14"  "MIF"    "NAMPT"  "MDK"    "FN1"    "COL1A1" "COL1A2"
 [9] "COL4A1" "COL4A2" "COL6A1" "COL6A2" "COL6A3" "LAMB1"  "APP"    "CDH5"  
[17] "CLDN11" "ESAM"   "JAM3"   "DLK1"   "JAG1"   "PECAM1" "GJA5"   "GJA1"  
[25] "DHCR24" "ITGA9"  "ITGB1"  "ACKR1"  "SDC2"   "LRP1"   "NCL"    "CD44"  
[33] "RORA"   "CD74"   "CDH5"   "CLDN11" "ESAM"   "JAM3"   "NOTCH3" "PECAM1"
[41] "VCAM1"  "GJA5"   "GJA1"   "ITGA5"  "ITGA9"  "ITGAV"  "ITGB1"

In [14]:

# We then visualize the upgulated and down-regulated signaling ligand-receptor pairs using bubble plot or chord diagram.
netVisual_bubble <- function (object, sources.use = NULL, targets.use = NULL, signaling = NULL, 
    pairLR.use = NULL, sort.by.source = FALSE, sort.by.target = FALSE, 
    sort.by.source.priority = TRUE, color.heatmap = c("Spectral", 
        "viridis"), n.colors = 10, direction = -1, thresh = 0.05, 
    comparison = NULL, group = NULL, remove.isolate = FALSE, 
    max.dataset = NULL, min.dataset = NULL, min.quantile = 0, 
    max.quantile = 1, line.on = TRUE, line.size = 0.2, color.text.use = TRUE, 
    color.text = NULL, title.name = NULL, font.size = 10, font.size.title = 10, 
    show.legend = TRUE, grid.on = TRUE, color.grid = "grey90", 
    angle.x = 90, vjust.x = NULL, hjust.x = NULL, return.data = FALSE) 
{
    color.heatmap <- match.arg(color.heatmap)
    if (is.list(object@net[[1]])) {
        message("Comparing communications on a merged object \n")
    }
    else {
        message("Comparing communications on a single object \n")
    }
    if (is.null(vjust.x) | is.null(hjust.x)) {
        angle = c(0, 45, 90)
        hjust = c(0, 1, 1)
        vjust = c(0, 1, 0.5)
        vjust.x = vjust[angle == angle.x]
        hjust.x = hjust[angle == angle.x]
    }
    if (length(color.heatmap) == 1) {
        color.use <- tryCatch({
            RColorBrewer::brewer.pal(n = n.colors, name = color.heatmap)
        }, error = function(e) {
            (scales::viridis_pal(option = color.heatmap, direction = -1))(n.colors)
        })
    }
    else {
        color.use <- color.heatmap
    }
    if (direction == -1) {
        color.use <- rev(color.use)
    }
    if (!is.null(pairLR.use)) {
        if (!is.data.frame(pairLR.use)) {
            stop("pairLR.use should be a data frame with a signle column named either 'interaction_name' or 'pathway_name' ")
        }
        else if ("pathway_name" %in% colnames(pairLR.use)) {
            pairLR.use$pathway_name <- as.character(pairLR.use$pathway_name)
        }
        else if ("interaction_name" %in% colnames(pairLR.use)) {
            pairLR.use$interaction_name <- as.character(pairLR.use$interaction_name)
        }
    }
    if (is.null(comparison)) {
        cells.level <- levels(object@idents)
        if (is.numeric(sources.use)) {
            sources.use <- cells.level[sources.use]
        }
        if (is.numeric(targets.use)) {
            targets.use <- cells.level[targets.use]
        }
        df.net <- subsetCommunication(object, slot.name = "net", 
            sources.use = sources.use, targets.use = targets.use, 
            signaling = signaling, pairLR.use = pairLR.use, thresh = thresh)
        df.net$source.target <- paste(df.net$source, df.net$target, 
            sep = " -> ")
        source.target <- paste(rep(sources.use, each = length(targets.use)), 
            targets.use, sep = " -> ")
        source.target.isolate <- setdiff(source.target, unique(df.net$source.target))
        if (length(source.target.isolate) > 0) {
            df.net.isolate <- as.data.frame(matrix(NA, nrow = length(source.target.isolate), 
                ncol = ncol(df.net)))
            colnames(df.net.isolate) <- colnames(df.net)
            df.net.isolate$source.target <- source.target.isolate
            # print('1')
            # print(unique(df.net.isolate$source.target))
            df.net.isolate$interaction_name_2 <- df.net$interaction_name_2[1]
            df.net.isolate$pval <- 1
            a <- stringr::str_split(df.net.isolate$source.target, 
                " -> ", simplify = T)
            df.net.isolate$source <- as.character(a[, 1])
            df.net.isolate$target <- as.character(a[, 2])
            df.net <- rbind(df.net, df.net.isolate)
        }
        df.net$pval[df.net$pval > 0.05] = 1
        df.net$pval[df.net$pval > 0.01 & df.net$pval <= 0.05] = 2
        df.net$pval[df.net$pval <= 0.01] = 3
        df.net$prob[df.net$prob == 0] <- NA
        df.net$prob.original <- df.net$prob
        df.net$prob <- -1/log(df.net$prob)
        idx1 <- which(is.infinite(df.net$prob) | df.net$prob < 
            0)
        if (sum(idx1) > 0) {
            values.assign <- seq(max(df.net$prob, na.rm = T) * 
                1.1, max(df.net$prob, na.rm = T) * 1.5, length.out = length(idx1))
            position <- sort(prob.original[idx1], index.return = TRUE)$ix
            df.net$prob[idx1] <- values.assign[match(1:length(idx1), 
                position)]
        }
        df.net$source <- factor(df.net$source, levels = cells.level[cells.level %in% 
            unique(df.net$source)])
        df.net$target <- factor(df.net$target, levels = cells.level[cells.level %in% 
            unique(df.net$target)])
        group.names <- paste(rep(levels(df.net$source), each = length(levels(df.net$target))), 
            levels(df.net$target), sep = " -> ")
        df.net$interaction_name_2 <- as.character(df.net$interaction_name_2)
        df.net <- with(df.net, df.net[order(interaction_name_2), 
            ])
        df.net$interaction_name_2 <- factor(df.net$interaction_name_2, 
            levels = unique(df.net$interaction_name_2))
        cells.order <- group.names
        df.net$source.target <- factor(df.net$source.target, 
            levels = cells.order)
        # print('2')
        # print(unique(df.net$source.target))
        df <- df.net
    }
    else {
        dataset.name <- names(object@net)
        df.net.all <- subsetCommunication(object, slot.name = "net", 
            sources.use = sources.use, targets.use = targets.use, 
            signaling = signaling, pairLR.use = pairLR.use, thresh = thresh)
        df.all <- data.frame()
        for (ii in 1:length(comparison)) {
            cells.level <- levels(object@idents[[comparison[ii]]])
            if (is.numeric(sources.use)) {
                sources.use <- cells.level[sources.use]
            }
            if (is.numeric(targets.use)) {
                targets.use <- cells.level[targets.use]
            }
            df.net <- df.net.all[[comparison[ii]]]
            df.net$interaction_name_2 <- as.character(df.net$interaction_name_2)
            df.net$source.target <- paste(df.net$source, df.net$target, 
                sep = " -> ")
            # print('3')
            # print(unique(df.net$source.target))
            source.target <- paste(rep(sources.use, each = length(targets.use)), 
                targets.use, sep = " -> ")
            source.target.isolate <- setdiff(source.target, unique(df.net$source.target))
            if (length(source.target.isolate) > 0) {
                df.net.isolate <- as.data.frame(matrix(NA, nrow = length(source.target.isolate), 
                  ncol = ncol(df.net)))
                colnames(df.net.isolate) <- colnames(df.net)
                df.net.isolate$source.target <- source.target.isolate
                # print('4')
                # print(unique(df.net.isolate$source.target))
                df.net.isolate$interaction_name_2 <- df.net$interaction_name_2[1]
                df.net.isolate$pval <- 1
                a <- stringr::str_split(df.net.isolate$source.target, 
                  " -> ", simplify = T)
                df.net.isolate$source <- as.character(a[, 1])
                df.net.isolate$target <- as.character(a[, 2])
                df.net <- rbind(df.net, df.net.isolate)
            }
            df.net$source <- factor(df.net$source, levels = cells.level[cells.level %in% 
                unique(df.net$source)])
            df.net$target <- factor(df.net$target, levels = cells.level[cells.level %in% 
                unique(df.net$target)])
            group.names <- paste(rep(levels(df.net$source), each = length(levels(df.net$target))), 
                levels(df.net$target), sep = " -> ")
            group.names0 <- group.names
            group.names <- paste0(group.names0, " (", dataset.name[comparison[ii]], 
                ")")
            if (nrow(df.net) > 0) {
                df.net$pval[df.net$pval > 0.05] = 1
                df.net$pval[df.net$pval > 0.01 & df.net$pval <= 
                  0.05] = 2
                df.net$pval[df.net$pval <= 0.01] = 3
                df.net$prob[df.net$prob == 0] <- NA
                df.net$prob.original <- df.net$prob
                df.net$prob <- -1/log(df.net$prob)
            }
            else {
                df.net <- as.data.frame(matrix(NA, nrow = length(group.names), 
                  ncol = 5))
                colnames(df.net) <- c("interaction_name_2", "source.target", 
                  "prob", "pval", "prob.original")
                df.net$source.target <- group.names0
                # print('5')
                # print(unique(df.net$source.target))
            }
            df.net$group.names <- as.character(df.net$source.target)
            df.net$source.target <- paste0(df.net$source.target, " (", dataset.name[comparison[ii]], ")")
            # print('6')
            # print(unique(df.net$source.target))
            df.net$dataset <- dataset.name[comparison[ii]]
            df.all <- rbind(df.all, df.net)
        }
        if (nrow(df.all) == 0) {
            stop("No interactions are detected. Please consider changing the cell groups for analysis. ")
        }
        idx1 <- which(is.infinite(df.all$prob) | df.all$prob < 
            0)
        if (sum(idx1) > 0) {
            values.assign <- seq(max(df.all$prob, na.rm = T) * 
                1.1, max(df.all$prob, na.rm = T) * 1.5, length.out = length(idx1))
            position <- sort(df.all$prob.original[idx1], index.return = TRUE)$ix
            df.all$prob[idx1] <- values.assign[match(1:length(idx1), 
                position)]
        }
        df.all$interaction_name_2[is.na(df.all$interaction_name_2)] <- df.all$interaction_name_2[!is.na(df.all$interaction_name_2)][1]
        df <- df.all
        df <- with(df, df[order(interaction_name_2), ])
        df$interaction_name_2 <- factor(df$interaction_name_2, 
            levels = unique(df$interaction_name_2))
        cells.order <- c()
        dataset.name.order <- c()
        for (i in 1:length(group.names0)) {
            for (j in 1:length(comparison)) {
                cells.order <- c(cells.order, paste0(group.names0[i], 
                  " (", dataset.name[comparison[j]], ")"))
                dataset.name.order <- c(dataset.name.order, dataset.name[comparison[j]])
            }
        }
        df$source.target <- factor(df$source.target, levels = cells.order)
        # print('7')
        # print(unique(df$source.target))
    }
    min.cutoff <- quantile(df$prob, min.quantile, na.rm = T)
    max.cutoff <- quantile(df$prob, max.quantile, na.rm = T)
    df$prob[df$prob < min.cutoff] <- min.cutoff
    df$prob[df$prob > max.cutoff] <- max.cutoff
    if (remove.isolate) {
        df <- df[!is.na(df$prob), ]
        line.on <- FALSE
    }
    if (!is.null(max.dataset)) {
        signaling <- as.character(unique(df$interaction_name_2))
        for (i in signaling) {
            df.i <- df[df$interaction_name_2 == i, , drop = FALSE]
            cell <- as.character(unique(df.i$group.names))
            for (j in cell) {
                df.i.j <- df.i[df.i$group.names == j, , drop = FALSE]
                values <- df.i.j$prob
                idx.max <- which(values == max(values, na.rm = T))
                idx.min <- which(values == min(values, na.rm = T))
                dataset.na <- c(df.i.j$dataset[is.na(values)], 
                  setdiff(dataset.name[comparison], df.i.j$dataset))
                if (length(idx.max) > 0) {
                  if (!(df.i.j$dataset[idx.max] %in% dataset.name[max.dataset])) {
                    df.i.j$prob <- NA
                  }
                  else if ((idx.max != idx.min) & !is.null(min.dataset)) {
                    if (!(df.i.j$dataset[idx.min] %in% dataset.name[min.dataset])) {
                      df.i.j$prob <- NA
                    }
                    else if (length(dataset.na) > 0 & sum(!(dataset.name[min.dataset] %in% 
                      dataset.na)) > 0) {
                      df.i.j$prob <- NA
                    }
                  }
                }
                df.i[df.i$group.names == j, "prob"] <- df.i.j$prob
            }
            df[df$interaction_name_2 == i, "prob"] <- df.i$prob
        }
    }
    if (remove.isolate) {
        df <- df[!is.na(df$prob), ]
        line.on <- FALSE
    }
    if (nrow(df) == 0) {
        stop("No interactions are detected. Please consider changing the cell groups for analysis. ")
    }
    if (!is.null(pairLR.use)) {
        interaction_name_2.order <- intersect(object@DB$interaction[pairLR.use$interaction_name, 
            ]$interaction_name_2, unique(df$interaction_name_2))
        df$interaction_name_2 <- factor(df$interaction_name_2, 
            levels = interaction_name_2.order)
    }
    df$source.target = droplevels(df$source.target, exclude = setdiff(levels(df$source.target), 
        unique(df$source.target)))
    # print('8')
    # print(unique(df$source.target))
    if (sort.by.target & !sort.by.source) {
        if (!is.null(targets.use)) {
            df$target <- factor(df$target, levels = intersect(targets.use, 
                df$target))
            df <- with(df, df[order(target, source), ])
            source.target.order <- unique(as.character(df$source.target))
            df$source.target <- factor(df$source.target, levels = source.target.order)
        }
    }
    if (sort.by.source & !sort.by.target) {
        if (!is.null(sources.use)) {
            df$source <- factor(df$source, levels = intersect(sources.use, 
                df$source))
            df <- with(df, df[order(source, target), ])
            source.target.order <- unique(as.character(df$source.target))
            df$source.target <- factor(df$source.target, levels = source.target.order)
        }
    }
    if (sort.by.source & sort.by.target) {
        if (!is.null(sources.use)) {
            df$source <- factor(df$source, levels = intersect(sources.use, 
                df$source))
            if (!is.null(targets.use)) {
                df$target <- factor(df$target, levels = intersect(targets.use, 
                  df$target))
            }
            if (sort.by.source.priority) {
                df <- with(df, df[order(source, target), ])
            }
            else {
                df <- with(df, df[order(target, source), ])
            }
            source.target.order <- unique(as.character(df$source.target))
            df$source.target <- factor(df$source.target, levels = source.target.order)
        }
    }
    df$source.target = as.character(df$source.target)
    # print(typeof(df$source.target))    
    # print(unique(df$source.target)[4])
    # print(typeof(unique(df$source.target)[4]))
    # print(df[is.na(df$source.target),])
    df = df[!is.na(df$source.target),]
    df$source.target = factor(df$source.target)
    g <- ggplot(df, aes(x = interaction_name_2, y =  source.target, 
        color = prob, size = pval)) + geom_point(pch = 16) + 
        theme_linedraw() + theme(panel.grid.major = element_blank()) + 
        theme(axis.text.x = element_text(angle = angle.x, hjust = hjust.x, 
            vjust = vjust.x), axis.title.x = element_blank(), 
            axis.title.y = element_blank()) + scale_x_discrete(position = "bottom")
    values <- c(1, 2, 3)
    names(values) <- c("p > 0.05", "0.01 < p < 0.05", "p < 0.01")
    g <- g + scale_radius(range = c(min(df$pval), max(df$pval)), 
        breaks = sort(unique(df$pval)), labels = names(values)[values %in% 
            sort(unique(df$pval))], name = "p-value")
    if (min(df$prob, na.rm = T) != max(df$prob, na.rm = T)) {
        g <- g + scale_colour_gradientn(colors = colorRampPalette(color.use)(99), 
            na.value = "white", limits = c(quantile(df$prob, 
                0, na.rm = T), quantile(df$prob, 1, na.rm = T)), 
            breaks = c(quantile(df$prob, 0, na.rm = T), quantile(df$prob, 
                1, na.rm = T)), labels = c("min", "max")) + guides(color = guide_colourbar(barwidth = 0.5, 
            title = "Commun. Prob."))
    }
    else {
        g <- g + scale_colour_gradientn(colors = colorRampPalette(color.use)(99), 
            na.value = "white") + guides(color = guide_colourbar(barwidth = 0.5, 
            title = "Commun. Prob."))
    }
    g <- g + theme(text = element_text(size = font.size), plot.title = element_text(size = font.size.title)) + 
        theme(legend.title = element_text(size = 8), legend.text = element_text(size = 6))
    if (grid.on) {
        if (length(unique(df$source.target)) > 1) {
            g <- g + geom_vline(xintercept = seq(1.5, length(unique(df$source.target)) - 
                0.5, 1), lwd = 0.1, colour = color.grid)
        }
        if (length(unique(df$interaction_name_2)) > 1) {
            g <- g + geom_hline(yintercept = seq(1.5, length(unique(df$interaction_name_2)) - 
                0.5, 1), lwd = 0.1, colour = color.grid)
        }
    }
    if (!is.null(title.name)) {
        g <- g + ggtitle(title.name) + theme(plot.title = element_text(hjust = 0.5))
    }
    if (!is.null(comparison)) {
        if (line.on) {
            xintercept = seq(0.5 + length(dataset.name[comparison]), 
                length(group.names0) * length(dataset.name[comparison]), 
                by = length(dataset.name[comparison]))
            g <- g + geom_vline(xintercept = xintercept, linetype = "dashed", 
                color = "grey60", size = line.size)
        }
        if (color.text.use) {
            if (is.null(group)) {
                group <- 1:length(comparison)
                names(group) <- dataset.name[comparison]
            }
            if (is.null(color.text)) {
                color <- ggPalette(length(unique(group)))
            }
            else {
                color <- color.text
            }
            names(color) <- names(group[!duplicated(group)])
            color <- color[group]
            dataset.name.order <- levels(df$source.target)
            dataset.name.order <- stringr::str_match(dataset.name.order, 
                "\\(.*\\)")
            dataset.name.order <- stringr::str_sub(dataset.name.order, 
                2, stringr::str_length(dataset.name.order) - 
                  1)
            xtick.color <- color[dataset.name.order]
            g <- g + theme(axis.text.x = element_text(colour = xtick.color))
        }
    }
    if (!show.legend) {
        g <- g + theme(legend.position = "none")
    }
    if (return.data) {
        return(list(communication = df, gg.obj = g))
    }
    else {
        return(g)
    }
}
all_celltypes = unique(cch@idents[[1]])
setwd(fig.dir)
for (i in 1:length(all_celltypes)) {  # length(all_celltypes)
    print(all_celltypes[i])
    f1 = paste0(samples, '_deSig_as_source_', all_celltypes[i], '_up.pdf')
    # f2 = paste0(samples, '_deSig_as_source_', all_celltypes[i], '_down.pdf')
    f3 = paste0(samples, '_deSig_as_target_', all_celltypes[i], '_up.pdf')
    # f4 = paste0(samples, '_deSig_as_target_', all_celltypes[i], '_down.pdf')
    print('up')
    tryCatch({
        pairLR.use.up = net.up[, "interaction_name", drop = F]
        par(mar = c(1, 1, 1, 1))
        gg1 <- netVisual_bubble(cch, pairLR.use = pairLR.use.up, sources.use = all_celltypes[i], comparison = c(1, 2), angle.x=45, color.text.use = F, 
            title.name = paste0("Up-regulated signaling in ", names(cch.list)[2]))
        ggsave(filename = f1, plot=gg1, width = 10, height = 10)
    }, error = function(e) {
        print(e)
    })
    tryCatch({
        par(mar = c(1, 1, 1, 1))
        gg3 <- netVisual_bubble(cch, pairLR.use = pairLR.use.up, targets.use = all_celltypes[i], comparison = c(1, 2), angle.x=45, color.text.use = F,
            title.name = paste0("Up-regulated signaling in ", names(cch.list)[2]))
        ggsave(filename = f3, plot=gg1, width = 10, height = 10)
    }, error = function(e) {
        print(e)
    })
    # tryCatch({
    #     print('down')
    #     pairLR.use.down = net.down[, "interaction_name", drop = F]
    #     gg2 <- netVisual_bubble(cch, pairLR.use = pairLR.use.down, sources.use = all_celltypes[i], comparison = c(1, 2), angle.x=45, color.text.use = F, 
    #         title.name = paste0("Down-regulated signaling in ", names(cch.list)[2]))
    #     ggsave(filename = f2, plot=gg2, width = 10, height = 10)
    # }, error = function(e) {
    #     print(e)
    # })
    # tryCatch({
    #     gg4 <- netVisual_bubble(cch, pairLR.use = pairLR.use.down, targets.use = all_celltypes[i], comparison = c(1, 2), angle.x=45, color.text.use = F,
    #         title.name = paste0("Down-regulated signaling in ", names(cch.list)[2]))
    #     ggsave(filename = f4, plot=gg2, width = 10, height = 10)
    # }, error = function(e) {
    #     print(e)
    # })
}


[1] ST(RAD51B)
32 Levels: B_Plasma(IGKC) B_Plasma(IGLC2) B_Plasma(IGLL5) ... ST(ZFPM2)
[1] "up"


Comparing communications on a merged object 


Comparing communications on a merged object 




[1] ST(FNDC3B)
32 Levels: B_Plasma(IGKC) B_Plasma(IGLC2) B_Plasma(IGLL5) ... ST(ZFPM2)
[1] "up"


Comparing communications on a merged object 


Comparing communications on a merged object 




[1] ST(BEX2)
32 Levels: B_Plasma(IGKC) B_Plasma(IGLC2) B_Plasma(IGLL5) ... ST(ZFPM2)
[1] "up"


...

Comparing communications on a merged object 


Comparing communications on a merged object 




[1] ST(MAGI1)
32 Levels: B_Plasma(IGKC) B_Plasma(IGLC2) B_Plasma(IGLL5) ... ST(ZFPM2)
[1] "up"


Comparing communications on a merged object 


Comparing communications on a merged object 




[1] ERY_ENDO(HBB)
32 Levels: B_Plasma(IGKC) B_Plasma(IGLC2) B_Plasma(IGLL5) ... ST(ZFPM2)
[1] "up"


Comparing communications on a merged object 


Comparing communications on a merged object 




In [15]:


all_celltypes = unique(c(net.up$pathway_name))
setwd(fig.dir)
for (i in 1:length(all_celltypes)) {  # length(all_celltypes)
    print(all_celltypes[i])
    f1 = paste0(samples, '_deSig_', all_celltypes[i], '_up.pdf')
    f2 = paste0(samples, '_deSig_', all_celltypes[i], '_down.pdf')
    print('up')
    if (length(net.up)>=1) {
        tryCatch({
            pairLR.use.up = subset(net.up, pathway_name == all_celltypes[i])[, "interaction_name", drop = F]
            gg1 <- netVisual_bubble(cch, pairLR.use = pairLR.use.up, comparison = c(1, 2), angle.x=45, color.text.use = F, 
                title.name = paste0("Up-regulated signaling in ", names(cch.list)[2]))
            ggsave(filename = f1, plot=gg1, width = 17, height = 8)
        }, error = function(e) {
            print(e)
        })
    }

    # if (length(net.down)>=1) {
    #     tryCatch({
    #         print('down')
    #         pairLR.use.down = subset(net.down, pathway_name == all_celltypes[i])[, "interaction_name", drop = F]
    #         gg2 <- netVisual_bubble(cch, pairLR.use = pairLR.use.down, comparison = c(1, 2), angle.x=45, color.text.use = F, 
    #             title.name = paste0("Down-regulated signaling in ", names(cch.list)[2]))
    #         ggsave(filename = f2, plot=gg2, width = 17, height = 8)
    #     }, error = function(e) {
    #         print(e)
    #     })
    # }
}


[1] "CCL"
[1] "up"


Comparing communications on a merged object 




[1] "MIF"
[1] "up"


Comparing communications on a merged object 




[1] "VISFATIN"
[1] "up"


...

Comparing communications on a merged object 




[1] "VCAM"
[1] "up"


Comparing communications on a merged object 




[1] "GAP"
[1] "up"


Comparing communications on a merged object 




Visualize the enriched ligands, signaling,or ligand-receptor pairs in one condition compared to another condition using wordcloud

In [16]:
computeEnrichmentScore <- function (df, measure = c("ligand", "signaling", "LR-pair"), 
    species = c("mouse", "human"), color.use = NULL, color.name = "Dark2", 
    n.color = 8, scale = c(4, 0.8), min.freq = 0, max.words = 200, 
    random.order = FALSE, rot.per = 0, return.data = FALSE, seed = 1, 
    ...) 
{
    measure <- match.arg(measure)
    species <- match.arg(species)
    LRpairs <- as.character(unique(df$interaction_name))
    ES <- vector(length = length(LRpairs))
    for (i in 1:length(LRpairs)) {
        df.i <- subset(df, interaction_name == LRpairs[i])
        if (length(which(rowSums(is.na(df.i)) > 0)) > 0) {
            df.i <- df.i[-which(rowSums(is.na(df.i)) > 0), , 
                drop = FALSE]
        }
        ES[i] = mean(abs(df.i$ligand.logFC) * abs(df.i$receptor.logFC) * 
            abs(df.i$ligand.pct.2 - df.i$ligand.pct.1) * abs(df.i$receptor.pct.2 - 
            df.i$receptor.pct.1))
    }
    if (species == "mouse") {
        CellChatDB <- CellChatDB.mouse
    }
    else if (species == "human") {
        CellChatDB <- CellChatDB.human
    }
    df.es <- CellChatDB$interaction[LRpairs, c("ligand", "receptor", 
        "pathway_name")]
    df.es$score <- ES
    df.es.ensemble <- df.es %>% group_by(ligand) %>% summarize(total = sum(score))
    set.seed(seed)
    if (is.null(color.use)) {
        color.use <- RColorBrewer::brewer.pal(n.color, color.name)
    }
    freq <- df.es.ensemble$total
    # Check for missing values in freq vector
    if (any(is.na(freq))) {
    freq <- freq[!is.na(freq)]
    }

    # Compute the minimum frequency based on non-missing values
    min.freq <- ifelse(min.freq > max(freq), 0, min.freq)

    # Call the wordcloud function with the updated arguments
    # wordcloud::wordcloud(words = df.es.ensemble$ligand, freq = freq, 
    #                     min.freq = min.freq, max.words = max.words, 
    #                     scale = scale, random.order = random.order, 
    #                     rot.per = rot.per, colors = color.use, ...)
    wordcloud::wordcloud(words = df.es.ensemble$ligand, freq = freq, 
        min.freq = min.freq, max.words = max.words, scale = scale, 
        random.order = random.order, rot.per = rot.per, colors = color.use, 
        ...)
    if (return.data) {
        return(df.es.ensemble)
    }
}
# visualize the enriched ligands in the first condition
# pdf(paste0(samples, '.enrichment_down.pdf'), width = 10, height = 10)
# computeEnrichmentScore(net.down, species = 'human')
# dev.off()

# visualize the enriched ligands in the second condition
pdf(paste0(samples, '.enrichment_up.pdf'), width = 10, height = 10)
computeEnrichmentScore(net.up, species = 'human')
dev.off()

ERROR: Error in loadNamespace(x): there is no package called ‘wordcloud’


# Part IV: Visually compare cell-cell communication using Hierarchy plot, Circle plot or Chord diagram

Similar to the CellChat analysis of individual dataset, we can visualize the cell-cell communication network using Hierarchy plot, Circle plot or Chord diagram.

**Edge color/weight, node color/size/shape**: In all visualization plots, edge colors are consistent with the sources as sender, and edge weights are proportional to the interaction strength. Thicker edge line indicates a stronger signal. In the **Hierarchy plot and Circle plot**, circle sizes are proportional to the number of cells in each cell group. In the hierarchy plot, solid and open circles represent source and target, respectively. In the **Chord diagram**, the inner thinner bar colors represent the targets that receive signal from the corresponding outer bar. The inner bar size is proportional to the signal strength received by the targets. Such inner bar is helpful for interpreting the complex chord diagram. Note that there exist some inner bars without any chord for some cell groups, please just igore it because this is an issue that has not been addressed by [circlize](https://github.com/jokergoo/circlize) package.

In [25]:
setwd(fig.dir)
for (pathways.show in pathways) {
    f1 = paste0(samples, '_circleplot_', pathways.show,'.pdf')
    f2 = paste0(samples, '_regularheatmap_', pathways.show,'.pdf')
    tryCatch({
        weight.max <- getMaxWeight(frozen.list, slot.name = c("netP"), attribute = pathways.show) # control the edge weights across different datasets
        
        pdf(f1, width = 10, height = 10)
        par(mfrow = c(1,2), xpd=TRUE)
        for (i in 1:length(cch.list)) {
            netVisual_aggregate(cch.list[[i]], signaling = pathways.show, layout = "circle", edge.weight.max = weight.max[1], edge.width.max = 10, signaling.name = paste(pathways.show, names(cch.list)[i]))
        }
        dev.off()

        pdf(f2, width = 10, height = 10)
        par(mfrow = c(1,2), xpd=TRUE)
        for (i in 1:length(cch.list)) {
            ht = netVisual_heatmap(cch.list[[i]], signaling = pathways.show, color.heatmap = "Reds", title.name = paste(pathways.show, "signaling ",names(cch.list)[i]))
            ComplexHeatmap::draw(ht, ht_gap = unit(0.5, "cm"))
        }
        dev.off()
    }, error = function(e) {
        print(e)
    })

}


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 




Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a sin

# Part V: Compare the signaling gene expression distribution between different datasets

We can plot the gene expression distribution of signaling genes related to L-R pairs or signaling pathway using a Seurat wrapper function `plotGeneExpression`.

In [26]:
cch@meta$datasets = factor(cch@meta$datasets, levels = names(cch.list)) # set factor level
setwd(fig.dir)
for (pathways.show in pathways) {
    f1 = paste0(samples, '_violin_', pathways.show,'.pdf')
    # f2 = paste0(samples, '_regularheatmap_', pathways.show,'.pdf')
    tryCatch({
        weight.max <- getMaxWeight(frozen.list, slot.name = c("netP"), attribute = pathways.show) # control the edge weights across different datasets
        # pdf(f1, width = 10, height = 10)
        g = plotGeneExpression(cch, signaling = pathways.show, split.by = "datasets", colors.ggplot = T)
        # dev.off()
        ggsave(filename = f1, plot=g, width = 10, height = 10)
    }, error = function(e) {
        print(e)
    })
}


<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleError in Idents(object): could not find function "Idents">
<simpleErr